In [ ]:
!pip install crewai
!pip install PyPDF2
!pip install crewai_tools
!pip install requests

import os
import requests
from crewai import Agent, Task, Crew
from PyPDF2 import PdfReader
from crewai_tools import SerperDevTool, WebsiteSearchTool



In [ ]:

# Set up API keys
os.environ["SERPER_API_KEY"] =""
os.environ["OPENAI_API_KEY"] = ""

# Download the PDF file from the URL
pdf_url = 'https://cdn1.lalpathlabs.com/live/reports/WM17S.pdf'
local_pdf_path = 'WM17S.pdf'

response = requests.get(pdf_url)
with open(local_pdf_path, 'wb') as file:
    file.write(response.content)

# Extract text from specific pages of the PDF
pages_to_extract = [1, 3]
raw_text = ''
with open(local_pdf_path, 'rb') as file:
    reader = PdfReader(file)
    for page_num in pages_to_extract:
        page = reader.pages[page_num - 1]
        content = page.extract_text()
        if content:
            raw_text += content

# print(raw_text)
summary_text = raw_text[:1000]

# Define tools
search_tool = SerperDevTool()
web_search_tool = WebsiteSearchTool()

# Create Agents
blood_test_analyst = Agent(
    role='Blood Test Analyst',
    goal='Analyze the blood test report and summarize the findings.',
    backstory='A medical expert specializing in blood test analysis.',
    verbose=True,
    allow_delegation=False
)

article_researcher = Agent(
    role='Article Researcher',
    goal='Search for health articles based on blood test results.',
    backstory='An expert researcher proficient in finding health-related articles.',
    tools=[search_tool, web_search_tool],
    verbose=True,
    allow_delegation = False,
)

health_advisor = Agent(
    role='Health Advisor',
    goal='Provide health recommendations based on the articles found.',
    backstory='A health advisor with extensive knowledge in providing health advice.',
    verbose=True,
    allow_delegation = False,
)

# Define Tasks
analyze_blood_test_task = Task(
    description= f'You have to analyze the blood test report from "{raw_text}" blood report',
    expected_output='A summary of the blood test results.',
    agent=blood_test_analyst,
)

find_articles_task = Task(
    description='Search for health articles based on the blood test analysis.',
    expected_output='A list of relevant health articles with links.',
    agent=article_researcher,
    context=[analyze_blood_test_task]
)

provide_recommendations_task = Task(
    description='Provide health recommendations based on the articles found.',
    expected_output='Health recommendations with links to the articles.',
    agent=health_advisor,
    context=[find_articles_task]
)

# Create a Crew
crew = Crew(
    agents=[blood_test_analyst, article_researcher, health_advisor],
    tasks=[analyze_blood_test_task, find_articles_task, provide_recommendations_task],
    verbose=True
)

# Execute Tasks
crew.kickoff()